In [758]:
import json
import os

import matplotlib.pyplot as plt
import numpy as np

result_dir = os.path.join(os.getcwd(), "results")
results = os.listdir(result_dir)

def load(f):
    file_dir = os.path.join(result_dir, f, "results.json")
    with open(file_dir, "r") as f:

        results = json.load(f)
    return results

# create a function that captalizes the first letter of a word
def capitalize(string):
    return string[0].upper() + string[1:]

def load_from_query(query):
    results = []
    for f in os.listdir(result_dir):
        if f.find(query) != -1:
            return load(f)
    return None

def show(data, scale=1, labels = None):
    for i, d in enumerate(data):
        print("" if not labels else labels[i] + " " + "|".join([f"{x*scale if x else 0:.4f}" for x in d]))
        
cache_sizes = [5, 10, 30, 50, 80, 100]
NR = 3
Nr = 30

avg_delay = []
hit_ratio = []
success_ratio = []
labels = []

for cache in ["random","mcfed","avgfed","nocache",]:
    for delivery in ["random", "greedy", "drl", "norsu"]:
                if cache in ["random", "avgfed", "nocache"] and delivery not in [
                    "drl",
                    "norsu",
                ]:
                    continue
                if delivery == "norsu" and cache != "random":
                    continue
                
                labels.append(f"{capitalize(cache)}_{capitalize(delivery)}")
                sub_data1 = []
                sub_data2 = []
                sub_data3 = []
                for cache_size in cache_sizes:
                    query = f"{NR}_{cache_size}_{Nr}_{cache}_{delivery}"
                    result = load_from_query(query)
                    if result:
                        sub_data1.append(np.mean(result["round_avg_delay_tracking"]))
                        sub_data2.append(np.mean(result["round_hit_ratio_tracking"]))
                        sub_data3.append(np.mean(result["round_success_ratio_tracking"]))
                    else:
                        sub_data1.append(None)
                        sub_data2.append(None)
                        sub_data3.append(None)
                avg_delay.append(sub_data1)
                hit_ratio.append(sub_data2)
                success_ratio.append(sub_data3)

In [759]:
labels

['Random_Drl',
 'Random_Norsu',
 'Mcfed_Random',
 'Mcfed_Greedy',
 'Mcfed_Drl',
 'Avgfed_Drl',
 'Nocache_Drl']

In [760]:
show(avg_delay, 1000, labels)

Random_Drl 186.8583|0.0000|0.0000|0.0000|0.0000|0.0000
Random_Norsu 363.7028|0.0000|0.0000|0.0000|0.0000|0.0000
Mcfed_Random 229.2685|0.0000|0.0000|0.0000|0.0000|0.0000
Mcfed_Greedy 205.4945|0.0000|0.0000|0.0000|0.0000|0.0000
Mcfed_Drl 0.0000|0.0000|0.0000|0.0000|0.0000|0.0000
Avgfed_Drl 0.0000|0.0000|0.0000|0.0000|0.0000|0.0000
Nocache_Drl 186.9657|186.8629|186.9649|187.0036|186.9494|187.0589


In [761]:
show(hit_ratio)